In [13]:
from langchain_groq import ChatGroq

In [14]:
llm = ChatGroq(
    model = "llama-3.1-70b-versatile",
    temperature = 0,
    max_tokens= None,
    timeout= None,
    max_retries= 2,
    groq_api_key = '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'
)

In [15]:
response = llm.invoke("first person to land on moon ?")
print(response.content)

The first person to land on the Moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the Moon's surface on July 20, 1969, as part of the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the Moon.


In [16]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-45852?from=job%20search%20funnel")
page_data = loader.load().pop().page_content
print(content)

Apply for Retail Associate I

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsRetail StoresRetail Associate IGuangzhou, GuangdongBecome a Part of the NIKE, Inc. TeamNIKE, Inc. does more 

In [17]:
from langchain_core.prompts import PromptTemplate
# so the major data from the webscaped data has been formateed here

prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    the scraped text is from career's page of a website
    your job is to extract the job postings and return then in JSON format containing the
    following keys: 'role' , 'experience' ,'skills' and 'description'.
    Only return the valid JSON
    ### VALID JSON (NO PREAMBLE):
    """
)

chain_extract = prompt_extract | llm
res = chain_extract.invoke(input= {'page_data':page_data})
print(res.content)

```json
{
  "role": "Director, Supply Chain AI/ML Engineering",
  "experience": "10+ years of experience across multiple roles in developing productionized code in software or data engineering, machine learning or a related field",
  "skills": [
    "Machine learning",
    "Artificial intelligence",
    "Data science",
    "Software engineering",
    "ETL",
    "ML",
    "Analytics technologies such as Scikit-learn, Dask, Tensorflow, Kubeflow, Spark, EMR",
    "Agile frameworks",
    "Cloud computing stacks",
    "MLOps",
    "Code management workflows",
    "Infrastructure as code"
  ],
  "description": "Nike is looking for a seasoned Engineering leader who can lead and grow teams of machine learning engineers, data scientists, and software architects to deliver scalable machine learning and artificial intelligence solutions to customers across our business."
}
```


In [20]:
type(res.content)

# convert this text to a json parser 

from langchain_core.output_parsers import JsonOutputParser
json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Director, Supply Chain AI/ML Engineering',
 'experience': '10+ years of experience across multiple roles in developing productionized code in software or data engineering, machine learning or a related field',
 'skills': ['Machine learning',
  'Artificial intelligence',
  'Data science',
  'Software engineering',
  'ETL',
  'ML',
  'Analytics technologies such as Scikit-learn, Dask, Tensorflow, Kubeflow, Spark, EMR',
  'Agile frameworks',
  'Cloud computing stacks',
  'MLOps',
  'Code management workflows',
  'Infrastructure as code'],
 'description': 'Nike is looking for a seasoned Engineering leader who can lead and grow teams of machine learning engineers, data scientists, and software architects to deliver scalable machine learning and artificial intelligence solutions to customers across our business.'}

In [21]:
import pandas as pd

df = pd.read_csv('my_portfolio.csv')
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [26]:
import chromadb
import uuid

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name = 'portfolio')

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents = row['Techstack'],
                      metadatas = {'links':row["Links"]},
                      ids=[str(uuid.uuid4())])

In [43]:
# Perform the query and Extract metadata
links = collection.query(
    query_texts=["Experience in python", "Expertise in React Native"],
    n_results=2
).get('metadatas') if result else None
# Print the metadata
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-native-portfolio'},
  {'links': 'https://example.com/react-portfolio'}]]

In [39]:
job = json_res
job['skills']

['Machine learning',
 'Artificial intelligence',
 'Data science',
 'Software engineering',
 'ETL',
 'ML',
 'Analytics technologies such as Scikit-learn, Dask, Tensorflow, Kubeflow, Spark, EMR',
 'Agile frameworks',
 'Cloud computing stacks',
 'MLOps',
 'Code management workflows',
 'Infrastructure as code']

In [44]:
# Perform the query and Extract metadata
links = collection.query(
    query_texts=job['skills'],
    n_results=2
).get('metadatas') if result else None
# Print the metadata
print(links)

[[{'links': 'https://example.com/ml-python-portfolio'}, {'links': 'https://example.com/wordpress-portfolio'}], [{'links': 'https://example.com/ml-python-portfolio'}, {'links': 'https://example.com/magento-portfolio'}], [{'links': 'https://example.com/ml-python-portfolio'}, {'links': 'https://example.com/magento-portfolio'}], [{'links': 'https://example.com/ml-python-portfolio'}, {'links': 'https://example.com/java-portfolio'}], [{'links': 'https://example.com/ml-python-portfolio'}, {'links': 'https://example.com/android-tv-portfolio'}], [{'links': 'https://example.com/ml-python-portfolio'}, {'links': 'https://example.com/magento-portfolio'}], [{'links': 'https://example.com/ml-python-portfolio'}, {'links': 'https://example.com/ios-ar-portfolio'}], [{'links': 'https://example.com/ios-ar-portfolio'}, {'links': 'https://example.com/devops-portfolio'}], [{'links': 'https://example.com/xamarin-portfolio'}, {'links': 'https://example.com/full-stack-js-portfolio'}], [{'links': 'https://exampl

In [ ]:
# First we creating a prompt template for extracting data 
# now we create another prompt template to create a email

In [49]:
prompt_email = PromptTemplate.from_template(
    """ 
    ### JOB DESCRIPTION
    (job_description)

    ###INSTRUCTION:
    Your name is Ali, you are a buisness development executive at NextSolution inc. NextSolution is an AI & Software consultant
    organization dedicated to provide seamless integrationof buisness process through automated tools.
    Over our experience , we have empowered numerous enterprises with tailored solutions fostering scalibility,
    procces optimization, cost reduction and heightened overall efficiency.
    your job is to write a cold email to client regarding the job mentioned above describing the capability of mentioned 
    above in a fulfilling their needs
    Also add the most relevant ones from the following links to showcase NextSolution's portfolio:{link_list}
    Remember you are Ali, BDE at NextSolution.
    Do not provide a preamble
    ###EMAIL (NO PREAMBLE)
    """
)

chain_extract = prompt_email | llm
res = chain_extract.invoke(input= {'job_description':str(job) , "link_list":links})
print(res.content)

Subject: Unlock Seamless Business Process Integration with NextSolution

Dear [Client],

I hope this email finds you well. As a Business Development Executive at NextSolution Inc., I am excited to introduce our organization and explore potential collaboration opportunities that can drive your business forward.

At NextSolution, we specialize in providing AI and software consulting services that enable seamless integration of business processes through automated tools. Our expertise lies in empowering enterprises with tailored solutions that foster scalability, process optimization, cost reduction, and heightened overall efficiency.

Our portfolio showcases a wide range of successful projects, including:

* Machine Learning and Python solutions: https://example.com/ml-python-portfolio
* Magento e-commerce solutions: https://example.com/magento-portfolio
* DevOps and full-stack JavaScript solutions: https://example.com/devops-portfolio and https://example.com/full-stack-js-portfolio

The